## Проверка однородности. Критерий хи-квадрат

Посмотрим, как применять критерии хи-квадрат для проверки однородности независимых выборок.

In [2]:
# как обычно, начнем с импорта нужных библиотек
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

/Users/a19028558/Desktop/transfer_accounts/akovyaz/learning_myself/env_school/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# и зафиксируем seed
np.random.seed(123)

**Задача.** Перед вами результаты медицинских исследований. Из 1500 мужчин, испытывающих лекарство, выздоровели 700, из 210 не принимавших выздоровели 80. Из 220 принимавших женщин — 150, из 680 не принимавших — 400. Проверить, влияет ли лекарство на мужчин? На женщин? На людей обоих полов?

Запишем аккуратно данные. Пусть $A$ — принимавшие лекарство, $\overline{A}$ — не принимавшие лекарство, $B$ — выздоровевшие, $\overline{B}$ — не выздоровевшие.

<!--<img width="60%" src="pics/pic2.png">-->
<table>
<tr><td>
    
|Мужчины| $B$ |  $\overline{B}$|
|--|--|--|
|$A$| 700 | 800 |
|$\overline{A}$| 80 | 130 |

</td><td>
    
|Женщины| $B$ |  $\overline{B}$|
|--|--|--|
|$A$| 150 | 70 |
|$\overline{A}$| 400 | 280 |

</td><td>

|Вместе| $B$ |  $\overline{B}$|
|--|--|--|
|$A$| 850 | 870 |
|$\overline{A}$| 480 | 410 |

</td></tr> </table>

In [4]:
# перепишем данные 

men = np.array([[700,800],[80,130]])
women = np.array([[150,70],[400,280]])
both = men + women

In [7]:
men

array([[700, 800],
       [ 80, 130]])

### Есть ли эффект от лекарства у мужчин? 

Заметим, что среди принимавших лекарство мужчин доля выздоровевших больше, чем среди мужчин, не принимавших лекарство:

$$\frac{700}{700 + 800} \approx 0.467 \qquad > \qquad 0.381 \approx \frac{80}{80 + 130}.$$
  
Проверим, значимо ли это различие.

In [ ]:
# можно реализовать криетрий вручную 

n = np.sum(men) # количество испытуемых
n1,n2 = np.sum(men,axis=1) # количество испытуемых, принимавших и не принимавших лекарство
p = np.sum(men,axis=0)/n # вероятности попасть в (B) и (not B)
exmen = np.array([p*n1,p*n2]) # ожидаемые количества в каждой ячейке

statistic = np.sum((men - exmen)**2/exmen)
pvalue = 1-stats.chi2.cdf(statistic, df=1)

print("statistic = ", statistic)
print("p-value = ", pvalue)

In [6]:
# второй спобоб реализации 

stats.chi2_contingency(men, correction=False)

(5.455512229705774,
 0.019506684444057915,
 1,
 array([[684.21052632, 815.78947368],
        [ 95.78947368, 114.21052632]]))

Критерий хи-квадрат признал различие значимым на уровне $\approx 2\%$.

### Есть ли эффект от лекарства у женщин? 

Аналогично, среди принимавших лекарство женщин доля выздоровевших больше, чем среди женщин, не принимавших лекарство:

$$\frac{150}{150 + 70} \approx 0.682 \qquad > \qquad 0.588 \approx \frac{400}{400 + 280}.$$
  
Проверим, значимо ли это различие.

In [8]:
stats.chi2_contingency(women)

(5.737976595596912,
 0.016601907318792426,
 1,
 array([[134.44444444,  85.55555556],
        [415.55555556, 264.44444444]]))

Критерий хи-квадрат признал различие значимым на уровне $\approx 2\%$.

### Есть ли эффект от лекарства у мужчин и женщин вместе? 

Как это ни странно, из таблицы с объединенными результатами следует, что доля выздоровевших больше среди тех людей, которые лекарство **не принимали**:

$$\frac{850}{850+870} \approx 0.494 \qquad < \qquad 0.539 \approx \frac{480}{480+410}.$$
  
Проверим, значимо ли это различие.

In [9]:
stats.chi2_contingency(both)

(4.603156848108104,
 0.031913140873073946,
 1,
 array([[876.47509579, 843.52490421],
        [453.52490421, 436.47509579]]))

Критерий хи-квадрат признал различие значимым на уровне $\approx 3.2\%$.

### Причина парадокса

Причина парадокса заключается в непропорциональном представительстве в разных категориях: мужчины выздоравливают
хуже, но лекарство испытывалось в основном на них.

На абстрактном уровне речь идет о следующем: из неравенств
    
$$\frac{A_1}{A_1+B_1} \ge \frac{C_1}{C_1+D_1}\quad\text{и}\quad \frac{A_2}{A_2+B_2} \ge \frac{C_2}{C_2+D_2}$$

иногда делается поспешный вывод о справедливости неравенства

$$\frac{A_1+A_2}{A_1+B_1 + A_2+B_2} \ge \frac{C_1+C_2}{C_1+D_1+C_2+D_2}$$

к чему нет никаких предпосылок.

Г. Секей пишет: «Аналогично, новое лекарство может оказаться эффективным в каждом из десяти различных госпиталей, но объединение результатов укажет на то, что это лекарство либо бесполезно, либо вредно».

Кроме того, число мужчин (210), не принимавших лекарство, недостаточно велико: согласно таблице, приведенной в книге
Дж. Флейс «Статистические методы для изучения таблиц долей и пропорций», вероятность ошибки II рода, для таких данных
равна $50\%$. Чтобы обеспечить ошибку второго рода $10\%$, необходимо иметь не менее 475 пациентов в этой категории.

Этот пример показывает, что к выводам, основанным на применении критерия хи-квадрат, следует относиться с осторожностью.